In [1]:
from bls_py import (aggregation_info
    ,bls
    ,bls12381
    ,ec
    ,fields
    ,fields_t
    ,fields_t_c
   , keys
   , pairing
   , signature
    ,tdata
    ,tests
    ,threshold
    ,util)

from random import randint

We will work in the G1 (Fq) group since public keys are in the G1 group.

In [2]:
G = ec.generator_Fq()
Q = ec.default_ec.q

In [3]:
def Hp(point: bls.AffinePoint) -> bls.AffinePoint:
    return ec.hash_to_point_Fq(point.serialize())

def Hs(*xs):
    h = b''
    for x in xs:
        if type(x) == str:
            b = bytes(x, encoding="utf-8")
        elif type(x) == bls.AffinePoint:
            b = x.serialize()
        else:
            b = bytes(x)
        h = ec.hash256(h + b)
    return int.from_bytes(h, "big")

In [4]:
N = 3
m = "important message"

x0 = bls.PrivateKey.from_seed("0")
x1 = bls.PrivateKey.from_seed("1")
x2 = bls.PrivateKey.from_seed("2")
P0 = x0.get_public_key()
P1 = x1.get_public_key()
P2 = x2.get_public_key()

P = [P0.value.to_affine(),
     P1.value.to_affine(),
     P2.value.to_affine()]

x = x0
s = 0

In [5]:
# Key Image
I = x.value * Hp(P[s])

In [6]:
# Prepare to produce ring sig.

q = [randint(0, Q) for _ in range(N)]
w = [randint(0, Q) for _ in range(N)]

In [7]:
L = [
    q[i] * G if i == s else
    q[i] * G + w[i] * P[i]
    for i in range(N)
]
R = [
    q[i] * Hp(P[i]) if i == s else
    q[i] * Hp(P[i]) + w[i] * I
    for i in range(N)
]

In [8]:
C = Hs(m, *L, *R)

In [9]:
c = [0 if i == s else w[i] for i in range(N)]
c[s] = (C - sum(c)) % Q

r = [
    (q[s] - c[s] * x.value) % Q if i == s else
    q[i]
    for i in range(N)
]

In [31]:
sum(c) % Q

32842339959732529955833948762244812645724875992095776656162352646762978716171

In [32]:
C

32842339959732529955833948762244812645724875992095776656162352646762978716171

In [11]:
sig = (I, c, r)

In [26]:
# Now to verify

def verify(I, c, r, P, m):
    print("I", I)
    print("c", c)
    print("r", r)
    print("P", P)
    print("m", m)
    Lprime = [r[i] * G + c[i] * P[i] for i in range(len(P))]
    
    print([l == lp for (l, lp) in zip(L, Lprime)])
    
    Rprime = [r[i] * Hp(P[i]) + c[i] * I for i in range(len(P))]
    
    print([r == rp for (r, rp) in zip(R, Rprime)])
    sum_c = sum(c) % Q
    h_c = Hs(m, *Lprime, *Rprime)
    print("sum_c", sum_c)
    print("h_c", h_c)
    return sum_c == h_c

In [27]:
verify(*sig, P, m)

I AffinePoint(x=Fq(Q, 0xe4421..61a5a), y=Fq(Q, 0xd7ce2..44879), i=False)

c [1125997488476211858511153327728609958923316969538730796847623877070597838312028946851960216351034579922724709138618, 3088767996834369780648217248249742780841416208334504498983868598790411062548009789258983751799733832628253217700437, 3790053625132753147676209075493455573381874801964513004788457745149299212767361868766527066763425268171573596996690]
r [650686533407452148660561722754953372596131871766904211953056185071040969372571303701490367261854317731816863669703, 1025842924542166914275044691351095004775421990709006646643524831001908489386427913164968999420180774548982465844095, 1364942380301356537848717536294742811640268201611109232596727768186090974179958900948024948412545095054374563498259]
P [AffinePoint(x=Fq(Q, 0x7e9788a446bd70fecaba764b2791a66573aa53197463ececba089625b7f506fe95e223bc2df2a4bf1bbeb859b29dd08), y=Fq(Q, 0x1715f33b8971c727abb7f7e784672e8e057c8870ebe3b2eb0bf80498d956b6ad19d94599efbb169b78a717

False

In [14]:
P[0].to_affine()

AttributeError: 'AffinePoint' object has no attribute 'to_affine'